In [1]:
import os
# os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');
# os.system('cp snorkel.db\ candidates snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.matchers import RegexMatchSpan, Union

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')

parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)
temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]', longest_match_only=False)

In [4]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

# part_ngrams = OmniNgramsPart(parts_by_doc=parts_by_doc, n_max=3)
part_ngrams = OmniNgrams()
temp_ngrams = OmniNgramsTemp(n_max=3)

In [5]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, 
                        [part_ngrams, temp_ngrams], 
                        [parts_matcher, temp_matcher])

In [6]:
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance

corpus = get_ORM_instance(Corpus, session, 'Hardware')

In [7]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 50 s, sys: 746 ms, total: 50.8 s
Wall time: 51.5 s
Candidate Set (Hardware Candidates) contains 11914 Candidates


In [8]:
# from snorkel.models import CandidateSet
# from snorkel.utils import get_ORM_instance
# train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
# dev = get_ORM_instance(CandidateSet, session, 'Hardware Development Candidates')
# candidates = set()
# for c in train:
#     candidates.add(c)
# for c in dev:
#     candidates.add(c)
# print len(train)
# print len(dev)
# print len(candidates)

In [9]:
from snorkel.lf_helpers import get_aligned_ngrams
def LF_aligned_temp_word(c):
    ngrams = list(get_aligned_ngrams(c.temp))
    return 1 if ('storage' in ngrams or
                 'temperature' in ngrams or
                 'tstg' in ngrams or
                 'ts' in ngrams or
                 'stg' in ngrams) else 0
#     if ((c[0].parent.document.name).upper(),
#             (c[0].get_span()).upper(),
#             (''.join(c[1].get_span().split())).upper()) == ('RECTS01158-1', 'BC807-25', '-55'):
#         import pdb; pdb.set_trace()

In [10]:
%%time
from snorkel.utils import ProgressBar
pb = ProgressBar(len(candidates))
neg = set()
pos = set()
for i, c in enumerate(candidates):
    pb.bar(i)
    if LF_aligned_temp_word(c):
        pos.add(c)
pb.close()

[========================================] 100%

CPU times: user 1min 29s, sys: 1min 12s, total: 2min 42s
Wall time: 2min 43s


In [11]:
print len(pos)

3775


In [12]:
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance
from hardware_utils import entity_level_f1

tp = pos
fp = set()
tn = set()
fn = set()

corpus = get_ORM_instance(Corpus, session, 'Hardware Training')
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 0.483
Corpus Recall    0.827
Corpus F1        0.609
----------------------------------------
TP: 554 | FP: 594 | FN: 116



In [13]:
from pprint import pprint
pprint(FN)

set([('112823', 'BC547B', '-55'),
     ('112823', 'BC547C', '-55'),
     ('BC337', 'BC337-16', '-55'),
     ('BC337', 'BC337-25', '-55'),
     ('BC337', 'BC337-40', '-55'),
     ('BC337', 'BC338-25', '-55'),
     ('BC337-D', 'BC337-025G', '-55'),
     ('BC337-D', 'BC337-040G', '-55'),
     ('BC337-D', 'BC337-25RL1G', '-55'),
     ('BC337-D', 'BC337-25RLRAG', '-55'),
     ('BC337-D', 'BC337-25ZL1G', '-55'),
     ('BC337-D', 'BC337-40RL1G', '-55'),
     ('BC337-D', 'BC337-40ZL1G', '-55'),
     ('BC546-BC548C(TO-92)', 'BC547A', '-55'),
     ('BC546-BC548C(TO-92)', 'BC547B', '-55'),
     ('BC546-BC548C(TO-92)', 'BC547C', '-55'),
     ('BC546-BC548C(TO-92)', 'BC548A', '-55'),
     ('BC546-BC548C(TO-92)', 'BC548B', '-55'),
     ('BC546-D', 'BC547B', '-55'),
     ('BC546-D', 'BC547C', '-55'),
     ('BC546A_SERIES_B14-521026', 'BC546B', '-65'),
     ('BC546A_SERIES_B14-521026', 'BC546C', '-65'),
     ('BC546A_SERIES_B14-521026', 'BC547A', '-65'),
     ('BC546A_SERIES_B14-521026', 'BC547C', '-6